# Classificando a base de [wine](https://archive.ics.uci.edu/ml/datasets/Wine) usando um KNN
Esse notebook faz parte do material de apoio do tutorial [Introdução ao Scikit-learn - Parte 2: iniciando um projeto](http://computacaointeligente.com.br/outros/intro-sklearn-part-2/)

In [1]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

## Carregando os dados 

In [3]:
wine_dataset = load_wine()
X = wine_dataset['data']
y = wine_dataset['target']
nome_das_classes = wine_dataset.target_names
descricao = wine_dataset['DESCR']
print(descricao)

['class_0' 'class_1' 'class_2']
.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

:Number of Instances: 178
:Number of Attributes: 13 numeric, predictive attributes and the class
:Attribute Information:
    - Alcohol
    - Malic acid
    - Ash
    - Alcalinity of ash
    - Magnesium
    - Total phenols
    - Flavanoids
    - Nonflavanoid phenols
    - Proanthocyanins
    - Color intensity
    - Hue
    - OD280/OD315 of diluted wines
    - Proline
    - class:
        - class_0
        - class_1
        - class_2

:Summary Statistics:

============================= ==== ===== ======= =====
                                Min   Max   Mean     SD
============================= ==== ===== ======= =====
Alcohol:                      11.0  14.8    13.0   0.8
Malic Acid:                   0.74  5.80    2.34  1.12
Ash:                          1.36  3.23    2.36  0.27
Alcalinity of Ash:            10.6  30.0    19.5   3.3
Magnesium:            

## Pre-processamento
- Dividir o dataset em treino e teste
- Normalizar os dados usando `MinMaxScaler`

In [4]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=32)
print(f"Tamanho do conjunto de treino: {len(X_treino)}")
print(f"Tamanho do conjunto de teste: {len(X_teste)}")

Tamanho do conjunto de treino: 124
Tamanho do conjunto de teste: 54


### Normalizando os dados

In [5]:
normalizador = MinMaxScaler()
normalizador.fit(X_treino)
X_treino_norm = normalizador.transform(X_treino)
X_teste_norm = normalizador.transform(X_teste)

## Configurando e treinando o KNN
- Número de vizinhos = 3

In [6]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_treino_norm, y_treino)
print(f"Acurácia de treinamento: {knn.score(X_treino_norm, y_treino)}")

Acurácia de treinamento: 0.967741935483871


### Explorando alguns métodos da classe `KNeighborsClassifier`

In [7]:
y_pred = knn.predict(X_teste_norm) # retorna a classe diretamente
y_pred_prob = knn.predict_proba(X_teste_norm) # retorna a probabilidade de cada classe
acc_teste = knn.score(X_teste_norm, y_teste)
print(f"Acurácia de teste: {acc_teste}")
print("Predições para cada amostra:")
k = 1
for l, p in zip(y_pred, y_pred_prob):
    print(f"- Amostra {k}: label = {l} | probabilidades = {p}")
    k+=1

Acurácia de teste: 0.9814814814814815
Predições para cada amostra:
- Amostra 1: label = 1 | probabilidades = [0. 1. 0.]
- Amostra 2: label = 1 | probabilidades = [0. 1. 0.]
- Amostra 3: label = 0 | probabilidades = [1. 0. 0.]
- Amostra 4: label = 2 | probabilidades = [0. 0. 1.]
- Amostra 5: label = 2 | probabilidades = [0. 0. 1.]
- Amostra 6: label = 0 | probabilidades = [1. 0. 0.]
- Amostra 7: label = 0 | probabilidades = [1. 0. 0.]
- Amostra 8: label = 2 | probabilidades = [0. 0. 1.]
- Amostra 9: label = 0 | probabilidades = [1. 0. 0.]
- Amostra 10: label = 0 | probabilidades = [1. 0. 0.]
- Amostra 11: label = 1 | probabilidades = [0. 1. 0.]
- Amostra 12: label = 2 | probabilidades = [0. 0. 1.]
- Amostra 13: label = 0 | probabilidades = [0.66666667 0.33333333 0.        ]
- Amostra 14: label = 0 | probabilidades = [1. 0. 0.]
- Amostra 15: label = 0 | probabilidades = [1. 0. 0.]
- Amostra 16: label = 2 | probabilidades = [0. 0. 1.]
- Amostra 17: label = 1 | probabilidades = [0.33333333

### Relatório de classificação
- Obtendo um relatório de classificação com informações de recall, precision, F1-score.
- Imprimir a matriz de confusão

In [8]:
relatorio = classification_report(y_teste, y_pred, target_names=nome_das_classes)
print("Relatório de classificação:")
print(relatorio)

Relatório de classificação:
              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00        22
     class_1       1.00      0.94      0.97        16
     class_2       0.94      1.00      0.97        16

    accuracy                           0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54



In [9]:
mat_conf = confusion_matrix(y_teste, y_pred)
print("Matriz de confusão:")
print(mat_conf)

Matriz de confusão:
[[22  0  0]
 [ 0 15  1]
 [ 0  0 16]]


# Empacotando o nosso modelo em um `pipeline`
Para fins de aprendizagem, vamos juntar todo o processo de pre-processamento e modelagem dentro de um pipeline. A principal utilidade é simplificar o código e também utilizar-lo para o busca de parâmetros. Como já foi discutido cada passo nas células anteriores, aqui vamos fazer tudo na mesma célula.

In [10]:
knn_pipeline = Pipeline(steps=[
  ("normalizacao", MinMaxScaler()),  
  ("KNN", KNeighborsClassifier(n_neighbors=3))
])
knn_pipeline.fit(X_treino, y_treino)
y_pred = knn_pipeline.predict(X_teste)
y_pred_prob = knn_pipeline.predict_proba(X_teste)
print(f"Acurácia de treinamento: {knn_pipeline.score(X_treino, y_treino)}")

Acurácia de treinamento: 0.967741935483871


# Buscando números de vizinhos utilizando `GridSearchCV`

In [11]:
param_busca={
  'KNN__n_neighbors': [3, 5, 7]
}
buscador = GridSearchCV(knn_pipeline, param_grid=param_busca)
buscador.fit(X, y)
print("Melhor K:", buscador.best_params_)

Melhor K: {'KNN__n_neighbors': 7}


In [12]:
import pandas as pd
pd.DataFrame.from_dict(buscador.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_KNN__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001325,0.000225,0.003473,0.000741,3,{'KNN__n_neighbors': 3},0.916667,0.944444,1.000000,1.0,0.914286,0.955079,0.038180,2
1,0.000952,0.000207,0.002402,0.000265,5,{'KNN__n_neighbors': 5},0.888889,0.944444,1.000000,1.0,0.914286,0.949524,0.044810,3
2,0.000724,0.000066,0.002088,0.000274,7,{'KNN__n_neighbors': 7},0.972222,0.944444,0.972222,1.0,0.971429,0.972063,0.017571,1


# Salvando o modelo
Na documentação oficial é descrito duas maneiras de persistir modelos da `sklearn`: utilizando `pickle` ou `joblib`. Porém, a mesma recomenda a `joblib` pois ela é mais eficiente para carregar arrays com muitos dados, que é o caso da maioria dos modelos.

Vamos iniciar salvando o nosso modelo empacotado dentro do `pipeline`

In [13]:
import joblib
joblib.dump(knn_pipeline, 'knn_pipeline.joblib') 

['knn_pipeline.joblib']

Para carregar, é igualmente fácil

In [14]:
knn_pipeline_carregado = joblib.load('knn_pipeline.joblib') 
y_pred_prob = knn_pipeline_carregado.predict_proba(X_teste)
print(f"Acurácia de treinamento: {knn_pipeline_carregado.score(X_treino, y_treino)}")

Acurácia de treinamento: 0.967741935483871
